# Game reviews

This notebook is where I perfrom some of the basics to scrape metacritic of it's data on video games. With this I can perform some analysis, create some vizualizations and do a study on one of my favorite subjects.

First let's import some libraries

In [238]:
import pandas as pd
import numpy as np
import requests
import bs4
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen
import matplotlib as plt
%matplotlib inline

Below are the URLs for both the detailed and condensed views. There are differences to how it's laid out, the detailed has additional data such as images and review excerpts/descriptions but is a little harder to wrangle so to get the important info such as title, platfrom, year, user and metascore I'm using the condensed view

In [4]:
#URL = "https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?sort=desc&view=detailed"
URL = "https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?view=condensed&sort=desc&page=0"

Metacritic only allows browsers to access so just had to put this little header so it thinks I'm a person accessing

In [183]:
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers = {'User-Agent':user_agent,} 
request = urllib.request.Request(URL,None,headers) 
response = urllib.request.urlopen(request)
data = response.read()

soup = BeautifulSoup(data, "html.parser")

In [64]:
# Just to check everything is in order
# print(soup)

In [18]:
test = soup.find('div', class_ = "product_wrap")

Here is the pattern of the chunks im interested in

In [19]:
test

<div class="product_wrap">
<div class="basic_stat product_title">
<a href="/game/nintendo-64/the-legend-of-zelda-ocarina-of-time">
                            The Legend of Zelda: Ocarina of Time
                                                            (N64)
                                                    </a>
</div>
<div class="basic_stat product_score brief_metascore">
<div class="metascore_w small game positive">99</div>
</div>
<div class="more_stats condensed_stats">
<ul class="more_stats">
<li class="stat product_avguserscore">
<span class="label">User:</span>
<span class="data textscore textscore_outstanding">9.1</span>
</li>
<li class="stat release_date full_release_date">
<span class="label">Release Date:</span>
<span class="data">Nov 23, 1998</span>
</li>
</ul>
</div>
</div>

### Debugging

In [191]:
games = soup.find('div', class_ = "product_wrap")

In [192]:
games

<div class="product_wrap">
<div class="basic_stat product_title">
<a href="/game/wii/mysims-skyheroes">
                            MySims SkyHeroes
                                                            (WII)
                                                    </a>
</div>
<div class="basic_stat product_score brief_metascore">
<div class="metascore_w small game mixed">67</div>
</div>
<div class="more_stats condensed_stats">
<ul class="more_stats">
<li class="stat product_avguserscore">
<span class="label">User:</span>
<span class="data textscore textscore_mixed">5.3</span>
</li>
<li class="stat release_date full_release_date">
<span class="label">Release Date:</span>
<span class="data">Sep 28, 2010</span>
</li>
</ul>
</div>
</div>

#### Scrape the title and platform

In [193]:
title_platform = games.find('a').text
title_platform = str(title_platform).replace("\n".join(["  "]), "").strip()

print(title_platform)

MySims SkyHeroes
(WII)


Theres a little formatting to do here so need to pull platform and title from each other

In [148]:
title_platform
title = title_platform[0:title_platform.find('\n')]
platform = title_platform[title_platform.find('\n')+2:len(title_platform)-1]
print(len(platform))


print(platform)
print(title)
platform


3
N64
The Legend of Zelda: Ocarina of Time


'N64'

In [123]:

\>>> s = 'gfgfdAAA1234ZZZuijjk'
>>> start = s.find('AAA') + 3
>>> end = s.find('ZZZ', start)
>>> s[start:end]

'1234'

#### Scrape scores

In [70]:
m_score = games.find('div', class_ = 'metascore_w').text
u_score = games.find('span', class_ = 'textscore').text

print(m_score, u_score)

99 9.1


#### Scrape release date

In [84]:
r_date = games.find_all('span', class_= 'data')[1].text
print(r_date)

Nov 23, 1998


## The scraping loop
Right now, this loop is doing just the first page. But there are 161 pages each with 100 reviews on them so an approx 16100 games in this df

It seems like it takes about 3.775 seconds on average per page. So multiplying that by 162 and dividing by 50 seems to mean that it will take about 10 minutes to fully scrape

In [168]:
(3.775 *162)/60

10.192499999999999

Note that this chunk timed out once, not sure why but in loading a new page it didnt load fully. Possibly because I left the screen but not sure..

In [197]:
%%time
#df = pd.DataFrame(columns=["Title", "Platform", "Metascore", "User Score", "Release Date"])

for i in range(0,162):
    URL = "https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?view=condensed&sort=desc&page="+str(i)
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers = {'User-Agent':user_agent,} 
    request = urllib.request.Request(URL,None,headers) 
    response = urllib.request.urlopen(request)
    data = response.read()

    soup = BeautifulSoup(data, "html.parser")

    for games in soup.find_all('div', class_ = "product_wrap", limit=100):

        # Extract title and platform
        title_platform = games.find('a').text
        title_platform = str(title_platform).replace("\n".join(["  "]), "").strip()

        title = title_platform[0:title_platform.find('\n')]
        platform = title_platform[title_platform.find('\n')+2:len(title_platform)-1]

        # Extract Metascore and User score
        m_score = games.find('div', class_ = 'metascore_w').text
        u_score = games.find('span', class_ = 'textscore').text

        # Extract release date
        r_date = games.find_all('span', class_= 'data')[1].text

        df = df.append({'Title':title, 'Platform':platform ,'Metascore':m_score, 'User Score':u_score, 'Release Date':r_date}, ignore_index = True)

AttributeError: 'NoneType' object has no attribute 'text'

Now the finished data frame

In [204]:
df.head()

,Title,Platform,Metascore,User Score,Release Date
0,The Legend of Zelda: Ocarina of Time,N64,99,9.1,"Nov 23, 1998"
1,Tony Hawk's Pro Skater 2,PS,98,7.4,"Sep 20, 2000"
2,Grand Theft Auto IV,PS3,98,7.5,"Apr 29, 2008"
3,SoulCalibur,DC,98,8.7,"Sep 8, 1999"
4,Grand Theft Auto IV,X360,98,7.9,"Apr 29, 2008"


Of course I dont want to lose this so just saved to csv and pickled it as well for an easy read

In [199]:
df.to_csv('meta_reviews.csv')
df.to_pickle('meta_reviews.pkl')

## Analysis

In [207]:
reviews = df

In [208]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16099 entries, 0 to 16098
Data columns (total 5 columns):
Title           16099 non-null object
Platform        16099 non-null object
Metascore       16099 non-null object
User Score      16099 non-null object
Release Date    16099 non-null object
dtypes: object(5)
memory usage: 628.9+ KB


I've realized this requires some cleaning still due to values such as 'tbd' in scores and the fact that it's not numeric. Also, need to clean up the dates to be able to split and filter by year/month

In [248]:
reviews['Release Date'] = pd.to_datetime(reviews['Release Date'])
reviews['Metascore'] = pd.to_numeric(reviews['Metascore'])
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16099 entries, 0 to 16098
Data columns (total 5 columns):
Title           16099 non-null object
Platform        16099 non-null object
Metascore       16099 non-null int64
User Score      16099 non-null object
Release Date    16099 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 628.9+ KB


In [261]:
#user_score = reviews['User Score']
#user_score = user_score.replace("tbd", 'Nan')
# pd.to_numeric(user_score)
user_score = reviews.groupby('User Score')['Title'].count()
reviews[reviews['User Score']=='tbd']

,Title,Platform,Metascore,User Score,Release Date
474,Madden NFL 2005,GC,90,tbd,2004-08-09
868,Tiger Woods PGA Tour 2005,GC,88,tbd,2004-09-20
924,Guacamelee! 2,XONE,87,tbd,2019-01-18
999,NFL 2K2,XBOX,87,tbd,2002-01-09
1146,NASCAR 2005: Chase for the Cup,XBOX,86,tbd,2004-08-31
1317,Moto Racer Advance,GBA,86,tbd,2002-12-05
1698,Dave Mirra Freestyle BMX 2,GBA,85,tbd,2001-11-25
1946,Pinball FX 2: Marvel Pinball - Vengeance and V...,X360,84,tbd,2011-12-13
2034,NASCAR Thunder 2003,XBOX,84,tbd,2002-09-19
2098,Virtua Tennis,GBA,83,tbd,2002-10-08


### Platform

In [228]:
reviews['Platform'].unique()

array(['N64', 'PS', 'PS3', 'DC', 'X360', 'WII', 'XONE', 'Switch', 'PS2',
       'PS4', 'GC', 'XBOX', 'PC', 'WIIU', 'GBA', '3DS', 'DS', 'VITA', 'PSP'], dtype=object)

In [244]:
platform = reviews.groupby('Platform')['Title'].count()
platform_reviews = reviews.groupby('Platform').aggregate({'Metascore':'max', 'User Score':'max'})
platform
platform_reviews

,User Score,Metascore
Platform,,
3DS,tbd,94
DC,tbd,98
DS,tbd,93
GBA,tbd,95
GC,tbd,97
N64,tbd,99
PC,tbd,96
PS,tbd,98
PS2,tbd,97


In [217]:
reviews['Release Date'] = pd.to_datetime(reviews['Release Date'])

In [218]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16099 entries, 0 to 16098
Data columns (total 5 columns):
Title           16099 non-null object
Platform        16099 non-null object
Metascore       16099 non-null object
User Score      16099 non-null object
Release Date    16099 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 628.9+ KB


2000